# TODO
    - Find a way to import modules using (relative) path 

In [ ]:
# Automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import ipywidgets as widgets

In [ ]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
from fastai import imports

In [ ]:
fastai = "../../FastAi/fastai"
fastai.

In [ ]:
import os
os.listdir(fastai)

In [ ]:
import sys
print(sys.path)

In [ ]:
sys.path.append("../../FastAi/fastai")

# TODO
> Download the pictures in notebook <br>
>> Use Keras folder structure with "train", "valid", "train/class1", ... 
<br> <br>
>> Rename the pictures <cr>
    
> Refactor the methods 

# Downloading the images

Downloading the images using a program from https://github.com/hardikvasa/google-images-download and running it through bash scripting locally in the notebook. More possible configurations, such as multiple searchwords, different combinations, etc., but this is enough for now. 

To run this method:
> Using pip
> <div class="alert alert-block alert-info">
\$ pip install google_images_download
</div> <br>
> Manually using CLI
<div class="alert alert-block alert-info">
> \$ git clone https://github.com/hardikvasa/google-images-download.git <br>
> \$ cd google-images-download && sudo python setup.py install
</div> <br>
> Manually using UI
> <div class="alert alert-block alert-info">
> Go to the repo on github ==> Click on *Clone or Download* ==> Click on *Download ZIP* and save it on your local disk. </div>

### The specified searchword will be a class, which in this case will be *female_face* and *male_face* 

In [ ]:
def download_pictures(searchword, form="jpg", lim=100, directory="data/"):
    ! googleimagesdownload --keywords $searchword --format $form --limit $lim --output_directory $directory
    src_path = directory + searchword
    dst_train_path = directory + "train/"
    dst_valid_path = directory + "valid/"
    counter = 1
    if not os.path.isdir(dst_train_path + searchword and dst_valid_path + searchword):
        train_search_folder = dst_train_path + searchword
        valid_search_folder = dst_valid_path + searchword
        print(train_search_folder)
        ! mkdir $train_search_folder
        ! mkdir $valid_search_folder
    for file in os.listdir(src_path):
        src = src_path + "/" + file
        dst_end = searchword + "/" + searchword + "." + str(counter) + "." + form
        train_ratio = lim*0.8 
        if(counter <= train_ratio):
            dst = dst_train_path + dst_end
        else:
            dst = dst_valid_path + dst_end
        os.rename(src, dst)
        counter = counter + 1
    delete_dir = directory + searchword
    ! rm -rf $delete_dir

In [ ]:
download_pictures("Female_face", lim=10)
download_pictures("Male_face", lim=10)

# Deep Learning

Utilizing a resizing method from FastAi - includes augmentation (side_on) and zooming 
    - Have the possibility of speeding up the training (smaller pictures) in the start
    - bs = 64 is usually preferred until Cuda-Out-Of-Memory Exception 

In [ ]:
import os 
PATH = "data/"
os.listdir(PATH)

In [ ]:
train_path = "train/"
valid_path = "valid/"
print(os.listdir(PATH + train_path))
print(os.listdir(PATH + valid_path))

Possible to get a nicer output of the items in directories? 

In [ ]:
class1_path = "Male_face/"
class2_path = "Female_face/"
print("Training folder: ")
print("Class1: " , os.listdir(PATH + train_path + class1_path)[:2])
print("Class2: " , os.listdir(PATH + train_path + class2_path)[:2])

print("Validation folder: ")
print("Class1: " , os.listdir(PATH + valid_path + class1_path)[:2])
print("Class2: " , os.listdir(PATH + valid_path + class2_path)[:2])

In [ ]:
files = os.listdir(f'{PATH}{train_path}{class1_path}')[:1]
files

In [ ]:
img = plt.imread(f'{PATH}{train_path}{class1_path}{files[0]}')
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
img[:4, :4]

## Uncomment if needing to resize images 

In [ ]:
#from sklearn import metrics 

#def get_data(sz, bs):
#   tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
#   data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)
#   return data if sz > 300 else data.resize(340, 'tmp');

In [ ]:
arch = resnet50
sz = 299 # One of two standards for ImageNet (224)
bs = 32 # 64 resulted in cuda-out-of-memory-error

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, ps=0.2)

## Choosing a Learning Rate

In [ ]:
learn.lr_find()

## Problems because of non-valid image types

> <div class="alert alert-block alert-info">
Can use module *imghdr* to verify image types
</div>

In [ ]:
import imghdr
imghdr.what(f'{PATH}{train_path}{class1_path}{files[0]}')

### Need to loop through datasets to remove invalid images 

> <div class="alert alert-block alert-info">
But first, interesting to see how much data we will be losing:
</div

In [ ]:
class1_train = PATH + train_path + class1_path
class2_train = PATH + train_path + class2_path
class1_valid = PATH + valid_path + class1_path
class2_valid = PATH + valid_path + class2_path

In [ ]:
print(class1_path[:-1] + " training files: " , len(os.listdir(PATH + train_path + class1_path)))
print(class2_path[:-1] + " training files: " , len(os.listdir(PATH + train_path + class2_path)))
print(class1_path[:-1] + " validation files: " , len(os.listdir(PATH + valid_path + class1_path)))
print(class2_path[:-1] + " validation files: " , len(os.listdir(PATH + valid_path + class2_path)))

In [ ]:
file_paths = [class1_train, 
              class2_train, 
              class1_valid, 
              class2_valid]
for path in file_paths:
    for files in os.listdir(path):
        if imghdr.what(path + files) != 'jpeg':
            os.remove(path + files)
            print("Deleting " + files + " in folder " + path)

In [ ]:
print(class1_path[:-1] + " training files: " , len(os.listdir(PATH + train_path + class1_path)))
print(class2_path[:-1] + " training files: " , len(os.listdir(PATH + train_path + class2_path)))
print(class1_path[:-1] + " validation files: " , len(os.listdir(PATH + valid_path + class1_path)))
print(class2_path[:-1] + " validation files: " , len(os.listdir(PATH + valid_path + class2_path)))

## New data set, so new training

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, ps=0.2)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(1e-2, 2, cycle_len=1)

In [ ]:
learn.unfreeze()
lr = np.array([1e-4, 1e-3, 1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), 0)

accuracy_np(probs, y)

## Overfitting

### When validation loss >> training 

> <div class="alert alert-block alert-info">
Increase dropout, unfreeze and using differential learning rates

In [ ]:
learn = ConvLearner.pretrained(arch, data, ps=0.5)
learn.unfreeze()

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), 0)

accuracy_np(probs, y)

### Analyzing Results

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)